# Práctica 2: Query DSL

En esta práctica vamos a profundizar más sobre las distintas consultas que podemos realizar sobre Elasticsearch.

Las sentencias de este notebook las ejecutaremos sobre Kibana en la sección Dev Tools. Para acceder a Kibana entra en la siguiente URL: http://127.0.0.1:5601

Para ilustrar los diferentes tipos de consulta vamos a generar un conjunto de documentos en Elasticsearch con los siguientes campos:

* title
* authors
* summary
* publish_date
* num_reviews
* publisher 

Para realizar esta insercción vamos a utilizar la Bulk API, una api que permite realizar varias operaciones simultáneas sobre los índices. 

Esta sentencia crea un índice con nombre bookdb_index y tipo book, e inserta documentos en dicho índice.

`POST /bookdb_index/book/_bulk
{ "index": { "_id": 1 }}
{ "title": "Elasticsearch: The Definitive Guide", "authors": ["clinton gormley", "zachary tong"], "summary" : "A distibuted real-time and scalable search and analytics engine", "publish_date" : "2015-02-07", "num_reviews": 20, "publisher": "oreilly" }
{ "index": { "_id": 2 }}
{ "title": "Taming Text: How to Find, Organize, and Manipulate It", "authors": ["grant ingersoll", "thomas morton", "drew farris"], "summary" : "organize text using approaches such as full-text search , proper name recognition, clustering, tagging, information extraction, summarization and analytics", "publish_date" : "2013-01-24", "num_reviews": 23, "publisher": "manning" }
{ "index": { "_id": 3 }}
{ "title": "Elasticsearch Guide in Action", "authors": ["radu gheorge", "matthew lee hinman", "timothy Gatsby"], "summary" : "build scalable search applications using Elasticsearch without having to do complex low-level programming or understand advanced data science algorithms", "publish_date" : "2015-12-03", "num_reviews": 18, "publisher": "manning" }
{ "index": { "_id": 4 }}
{ "title": "Solr in Action", "authors": ["trey grainger", "timothy Gatsby"], "summary" : "Comprehensive guide to implementing a scalable search engine using Apache Solr and Elasticsearch", "publish_date" : "2014-04-05", "num_reviews": 23, "publisher": "manning" }
`

## 1. Ejercicio: Insertar los datos anteriores y comprobar los campos generados

Para ver el mapping type utiliza la API: `<nombre_indice>/<tipo>/_mappings`

## 2. Ejercicio: Incluir los siguientes libros ejecutando la Api Bulk

**title**: The Great Gatsby, authors: F. Scott Fitzgerald
**summary**: In my younger and more vulnerable years my father gave me some advice that I've been turning over in my mind ever since
**publish_date**: 2009-02-07
**num_reviews**: 5
**publisher**: anagrama 

**title**: The Grapes of Wrath and Gatsby
**authors**: [John Steinbeck, Lucas Stuart] 
**summary**: "To the red country and part of the gray country of Oklahoma, the last rains came gently, and they did not cut the scarred earth."
**publish_date**: 2009-02-17
**num_reviews**: 90 
**publisher**: salamandra

**title**: Nineteen Eighty-Four
**authors**: George Orwell
**summary**: "It was a bright cold day in April, and the clocks were striking thirteen"
**publish_date**: 1995-06-17
**num_reviews**: 23 
**publisher**: salamandra

## 3. Match Query

Hay dos maneras de ejecutar una consulta básica de full_text (match query): 
1. Usando la Search Lite API, que espera que todos los parámetros de búsqueda se transmitan como parte de la URL.
2. o incluyendo la consulta en el body del JSON, lo que permite usar la complejidad deseada del lenguaje de consulta Elasticsearch DSL.

`GET /bookdb_index/book/_search?q=Elasticsearch (1)`

`GET /bookdb_index/book/_search (2)
{
    "query": {
        "multi_match" : {
            "query" : "Elasticsearch",
            "fields" : ["_all"]
        }
    }
}
`

**Nota**: multi_match se utiliza en lugar de “match” como una forma abreviada de ejecutar la misma consulta en varios campos. La propiedad fields especifica los campos a consultar y en este caso queremos consultar con todos los campos del documento.

### 3.1 Ejercicio: Ahora te toca a tí realizar una búsqueda full_text (todos los campos) que devuelva los documentos que tengan la palabra salamandra.

Ambas APIs también permiten especificar en qué campos desea buscar. Por ejemplo, para buscar libros con las palabras "In action" en el campo title:

`GET /bookdb_index/book/_search?q=title:in action`

Sin embargo, el full_text DSL da más flexibilidad a la hora de crear consultas más complejas y especificar cómo desea que los resultados se muestren. En el ejemplo siguiente, indicamos el número de resultados que queremos recuperar, desde dónde empezar (útil para la paginación), los campos de documento que queremos devolver y
highlights.

`POST /bookdb_index/book/_search
{
    "query": {
        "match" : {
            "title" : "in action"
        }
    },
    "size": 2,
    "from": 0,
    "_source": [ "title", "summary", "publish_date" ],
    "highlight": {
        "fields" : {
            "title" : {}
        }
    }
}`

### 3.2 Ejercicio: Realiza una búsqueda sobre el campo summary del texto que devuelva como máximo 3 resultados y sólo devuelva los campos title, summary y num_reviews. Queremos marcar los aciertos (highligts) en el campo summary.

Como hemos visto anteriormente, para consultar más de un campo del documento en una búsqueda (por ejemplo, buscando la misma cadena de consulta en el título y el resumen), se puede utilizar la consulta multi_match.

`POST /bookdb_index/book/_search
{
    "query": {
        "multi_match" : {
            "query" : "elasticsearch guide",
            "fields": ["title", "summary"]
        }
    }
}
`

### 3.3 Ejercicio: Busca el texto "Gatsby" en los campos title y author

Dado que estamos buscando en varios campos, es posible que desee aumentar las relevancia de un determinado campo. Por ejemplo aumentar las relevancia del campo de resumen por un factor de 3 para aumentar la importancia de encontrar el texto en dicho campo.

`POST /bookdb_index/book/_search
{
    "query": {
        "multi_match" : {
            "query" : "elasticsearch guide",
            "fields": ["title", "summary^3"]
        }
    },
    "_source": ["title", "summary", "publish_date"]
}
`

### 3.4. Ejercicio: Modifica la consulta del ejercicio 3.3 para dar más peso a la aparición en el campo título y que sólo aparezcan como resultado de la consulta los campos, title, summary y publisher

## 4. Boolean Query
Los operadores AND / OR / NOT pueden utilizarse para afinar nuestras consultas de búsqueda con el fin de proporcionar resultados más relevantes o específicos. Esto se implementa en la API de búsqueda como una consulta booleana. La consulta booleana acepta un parámetro must (equivalente a AND), un parámetro must_not (equivalente a
NOT) y un parámetro should (equivalente a OR).

`POST /bookdb_index/book/_search
{
    "query": {
        "bool": {
            "must": {
                "bool" : { "should": [
                            { "match": { "title": "Elasticsearch" }},
                            { "match": { "title": "Solr" }} 
                        ] }
            },
            "must_not": { "match": {"authors": "radu gheorge" }}
        }
    }
}
`
### 4.1 Ejercicio: Realiza una consulta que encuentre los libros en los que aparezca en el título: "Elasticsearch Guide", que no sean del autor Mathew, junto con los que tengan 23 reviews

## 5. Wildcards Queries y expresiones regulares

Las consultas de comodines le permiten especificar un patrón para que coincida en lugar del término completo. ? coincide con cualquier carácter y * coincide con cero o más caracteres.

`POST /bookdb_index/book/_search
{
    "query": {
        "wildcard" : {
            "authors" : "t*"
        }
    },
    "_source": ["title", "authors"],
    "highlight": {
        "fields" : {
            "authors" : {}
        }
    }
}
`

### 5.1 Ejercicio: Busca los libros cuyo autor empiece por g y termine en 'orge'

## 6. Match Phrase Query

En el notebook anterior vimos una manera de ejecutar match_phrase queries. 
Este tipo de consulta requiere que todos los términos de la cadena estén presentes en el documento, estén en el orden especificado en la cadena de consulta y estén cerca entre sí. De forma predeterminada, los términos deben estar exactamente uno al lado del otro, pero puede especificar el valor de slop que indica cuán distantes se pueden
permitir los términos.

`POST /bookdb_index/book/_search
{
    "query": {
        "multi_match" : {
            "query": "search engine",
            "fields": ["title", "summary"],
            "type": "phrase",
            "slop": 3
        }
    }
}
`

###  6.1 Ejercicio: Buscar los libros que tengan en el título o sumario la frase "Comprehensive scalable". Incluir el slop máximo para que aparezca un documento

## 7. Range query

Permite buscar valores entre rangos.

`POST /bookdb_index/book/_search
{
    "query": {
        "range" : {
            "num_reviews": {
                "gte": "20",
                "lte": 50
            }
        }
    },
    "_source" : ["title","publish_date","publisher"]
}
`
### 7.1 Ejercicio: Buscar los libros que han sido publicados en 2015

## 8. Filtered queries

Permite realizar filtros sobre una query realizada. Por ejemplo buscar los libros con un número de reviews entre 10 y 20

`POST /bookdb_index/book/_search
{
    "query": {
        "bool": {
            "must": {
                "match_all": {}
            },
            "filter": {
                "range": {
                    "num_reviews": {
                        "gte": 10,
                        "lte": 20
                    }
                }
            }
        }
    }
}
`

### 8.1  Ejercicio: Buscar los libros de timothy Gatsby de marzo de 2014